TODO:
- find a (much) smaller model for quicker training and experiments with model uploads.
- 

In [3]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn 
import torch
import numpy as np
from transformers import set_seed
from transformers.integrations import NeptuneCallback
import os

set_seed(1)

def tokenize_function(X):
    return tokenizer(X['sentence1'], X['sentence2'], truncation=True)

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

os.environ["NEPTUNE_PROJECT"] = 'common/huggingface-integration'
os.environ["NEPTUNE_API_TOKEN"] = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhMjI0YjQ1Yi1iMGY4LTRmNjgtOGRjZC05MzEyMjVkMDRlMzEifQ=="


Prepare dataset, tokenizer, and pretrained model.

In [5]:
raw_dataset = load_dataset("glue", "mrpc") 
#checkpoint = "distilbert-base-uncased"
checkpoint = "google/mobilebert-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

Reusing dataset glue (/home/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/847 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

  0%|          | 0/4 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
''' Exemplary run '''

training_args = TrainingArguments(
    "fast-mobilebert-mrpc", 
    evaluation_strategy="steps",
    eval_steps = 20,
    save_strategy="epoch",
    logging_steps=1, 
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    num_train_epochs=3, 
    report_to = None
    )

neptuneClbck = NeptuneCallback(name="my-coolest-run")

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset  = tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[neptuneClbck]
    )

trainer.train()

# Neptune run is created when calling trainer.train(), so we can only add custom parameter after:
neptuneClbck._neptune_run["parameters/custom_parameter"] = "Better defaults"
neptuneClbck._neptune_run["sys/tags"].add(["modified", "better defaults"])

neptuneClbck.stop_run()

The following columns in the training set  don't have a corresponding argument in `MobileBertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
/home/transformers/src/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 40
  Total train batch size (w. parallel, distributed & accumulation) = 40
  Gradient Accumulation steps = 1
  Total optimization steps = 276


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss,Validation Loss,Accuracy,F1
20,12764.608400,16998.111328,0.671569,0.793846
40,0.716700,1.042160,0.458333,0.440506
60,0.676400,0.675543,0.686275,0.813411
80,0.597400,0.558097,0.730392,0.827044
100,0.717300,0.489680,0.759804,0.837748
120,0.635600,0.462595,0.799020,0.858621
140,0.454600,0.457977,0.801471,0.862944
160,0.420300,0.429464,0.789216,0.862179
180,0.427900,0.396557,0.808824,0.872131
200,0.614600,0.388725,0.806373,0.871545


The following columns in the evaluation set  don't have a corresponding argument in `MobileBertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 40
Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/train/epoch. Invalid point: 20.0
The following columns in the evaluation set  don't have a corresponding argument in `MobileBertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 40
Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/train/epoch. Invalid point: 40.0
The following columns in the evaluation set  don't have a corresponding argument in `MobileBertForSequenceClassification.forward` and have been ignored: sen

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 38 operations to synchronize with Neptune. Do not kill this process.
Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/train/epoch. Invalid point: 276.0


All 38 operations synced, thanks for waiting!


In [4]:
neptuneClbck.stop_run()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.


All 1 operations synced, thanks for waiting!


In [4]:
from neptune.new import neptune


ModuleNotFoundError: No module named 'neptune'